In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
c = pd.read_csv('customer_join.csv')
ul = pd.read_csv('use_log.csv')

In [5]:
ul.isnull().sum(), c.isnull().sum()

(log_id         0
 customer_id    0
 usedate        0
 dtype: int64,
 customer_id         0
 name                0
 class               0
 gender              0
 start_date          0
 end_date         2842
 campaign_id         0
 is_deleted          0
 class_name          0
 price               0
 campaign_name       0
 mean                0
 median              0
 amax                0
 amin                0
 flag1               0
 cal_date            0
 mem_period          0
 dtype: int64)

클러스터 사용 : K-Means

- 군집개수 : 4개
- min, max, median, mean 변수는 월 이용 횟수와 관련된 변수 : 1~8 사이의 값을 가짐
- mem_period 단위가 너무 커서 scaling

In [7]:
cc = c[['mean','median','amin','amax','mem_period']]
cc

,mean,median,amin,amax,mem_period
0,4.833333,5.0,2,8,47.968131
1,5.083333,5.0,3,7,47.968131
2,4.583333,5.0,3,6,47.968131
3,4.833333,4.5,2,7,47.968131
4,3.916667,4.0,1,6,47.968131
...,...,...,...,...,...
4187,8.000000,8.0,8,8,1.544180
4188,10.000000,10.0,10,10,1.544180
4189,8.000000,8.0,8,8,1.544180
4190,11.000000,11.0,11,11,1.544180


In [10]:
scaler = StandardScaler()
scale_data = pd.DataFrame(scaler.fit_transform(cc), columns=['mean','median','amin','amax','mem_period'])
scale_data.head()

,mean,median,amin,amax,mem_period
0,-0.281206,-0.133676,-0.533620,0.081177,2.031859
1,-0.140545,-0.133676,-0.021149,-0.379928,2.031859
2,-0.421867,-0.133676,-0.021149,-0.841034,2.031859
3,-0.281206,-0.400393,-0.533620,-0.379928,2.031859
4,-0.796964,-0.667109,-1.046090,-0.841034,2.031859


In [12]:
km = KMeans(n_clusters=4, random_state=0)
cluster = km.fit(scale_data)

In [13]:
cc['cluster'] = cluster.labels_
cc

C:\Users\user\AppData\Local\Temp\ipykernel_16068\3554618193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc['cluster'] = cluster.labels_


,mean,median,amin,amax,mem_period,cluster
0,4.833333,5.0,2,8,47.968131,1
1,5.083333,5.0,3,7,47.968131,1
2,4.583333,5.0,3,6,47.968131,1
3,4.833333,4.5,2,7,47.968131,1
4,3.916667,4.0,1,6,47.968131,1
...,...,...,...,...,...,...
4187,8.000000,8.0,8,8,1.544180,0
4188,10.000000,10.0,10,10,1.544180,0
4189,8.000000,8.0,8,8,1.544180,0
4190,11.000000,11.0,11,11,1.544180,0


In [14]:
cc.rename(columns={'mean':'월 평균','median':'월별 중간값', 'amin':'월별 최소값', 'amax':'월별 최대값', 'mem_period':'회원기간'}, inplace=True)
cc

C:\Users\user\AppData\Local\Temp\ipykernel_16068\738254444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc.rename(columns={'mean':'월 평균','median':'월별 중간값', 'amin':'월별 최소값', 'amax':'월별 최대값', 'mem_period':'회원기간'}, inplace=True)


,월 평균,월별 중간값,월별 최소값,월별 최대값,회원기간,cluster
0,4.833333,5.0,2,8,47.968131,1
1,5.083333,5.0,3,7,47.968131,1
2,4.583333,5.0,3,6,47.968131,1
3,4.833333,4.5,2,7,47.968131,1
4,3.916667,4.0,1,6,47.968131,1
...,...,...,...,...,...,...
4187,8.000000,8.0,8,8,1.544180,0
4188,10.000000,10.0,10,10,1.544180,0
4189,8.000000,8.0,8,8,1.544180,0
4190,11.000000,11.0,11,11,1.544180,0


In [15]:
cc.groupby('cluster').count()

,월 평균,월별 중간값,월별 최소값,월별 최대값,회원기간
cluster,,,,,
0,846,846,846,846,846
1,1251,1251,1251,1251,1251
2,759,759,759,759,759
3,1336,1336,1336,1336,1336


그룹 3번에 해당하는 수가 1336명으로 가장 많고 그룹1, 그룹0, 그룹2 순으로 사람이 많음

In [18]:
cc.groupby('cluster').mean()

,월 평균,월별 중간값,월별 최소값,월별 최대값,회원기간
cluster,,,,,
0,8.054608,8.039598,6.160757,10.009456,7.809637
1,4.676091,4.670264,2.153477,7.231815,37.819727
2,3.049270,2.882740,1.652174,4.745718,10.147960
3,5.522518,5.373129,2.686377,8.743263,15.688428


그룹별 특징을 정의해서 프로모션할 때 사용 가능

### 차원 축소
- feature 5개 -> 2차원으로 줄이기